In [ ]:
# Columns test

import pandas as pd

# Function to load the data
def load_data(url):
    df = pd.read_excel(url)
    return df

# Function to clean the data
def clean_data(df):
    # Rename the 'Unnamed: 11' column to 'Fatal'
    df = df.rename(columns={'Unnamed: 11': 'Fatal', 'Species ': 'Species'})
    
    # Drop rows with null values in the specified columns
    df = df.dropna(subset=['Time', 'Species', 'Source', 'pdf'])
    
    # Drop duplicates
    df = df.drop_duplicates()
    
    return df

# URL of the file
url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

# Load the data
df_sa = load_data(url)

# Clean the data
df_sa = clean_data(df_sa)

# Get unique values of the specified columns
unique_time = df_sa['Time'].unique()
unique_species = df_sa['Species'].unique()
unique_source = df_sa['Source'].unique()
unique_pdf = df_sa['pdf'].unique()
unique_fatal = df_sa['Fatal'].unique()

print("Unique values in the 'Time' column:", unique_time)
print("Unique values in the 'Species' column:", unique_species)
print("Unique values in the 'Source' column:", unique_source)
print("Unique values in the 'pdf' column:", unique_pdf)
print("Unique values in the 'Fatal' column:", unique_fatal)


In [ ]:
# Test

df_sa = load_data(url)

# Print the column names
print(df_sa.columns)

In [ ]:
# Main Code

import pandas as pd
from functions import load_data, clean_data, style_table

def main():
    # Load and clean data
    url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
    df_sa = load_data(url)
    df_sa = clean_data(df_sa)

    # Ask the user if they want to display the table
    show_table = input("Show table? Y/N: ").strip().lower()

    if show_table == 'y':
        styled_df = style_table(df_sa)
        display(styled_df)
    
    display(df_sa.head(10))

if __name__ == "__main__":
    main()

In [ ]:
# Show graphics for null values

import matplotlib.pyplot as plt
import seaborn as sns

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
df_sa = load_data(url)
df_sa = clean_data(df_sa)
sns.heatmap(df_sa.isnull(), cbar=False)
plt.show()